In [2]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
device = 'cpu'

In [3]:
def gen_data(batch = 4):
    t = np.arange(0,torch.randint(3,15,(1,)),step=0.1)
    X = torch.zeros(batch,t.size,1)
    Y = torch.zeros(batch,2)
    for i in range(batch):
        a = torch.rand(1)*3 + 0.1
        b = torch.rand(1)*4 + 0.2
        #X[i,:,0] =  torch.tensor(t)
        X[i,:,0] =  a*np.sin(b*t)
        Y[i,0],Y[i,1] = a,b
    return X,Y

In [4]:
from torch.nn import TransformerEncoder, TransformerEncoderLayer
class TransformerRegression(nn.Module):
    def __init__(self, input_dim, output_dim, nhead, hidden_dim, num_layers):
        super(TransformerRegression, self).__init__()

        self.embedding = nn.Linear(input_dim, hidden_dim)
        encoder_layer = TransformerEncoderLayer(hidden_dim, nhead)
        self.encoder = TransformerEncoder(encoder_layer, num_layers)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(1, 0, 2)  # Изменяем форму для передачи в трансформер
        output = self.encoder(x)  # Применяем трансформер
        #print(output.shape)
        #output = output.mean(dim=0)  # Усредняем по временным шагам
        output = output[0,:,:]
        output = self.fc(output)  # Полносвязный слой для предсказания
        return output

In [5]:

input_dim = 1
output_dim = 2
nhead = 2#4
hidden_dim = 32
num_layers = 2#4

model = TransformerRegression(input_dim, output_dim, nhead, hidden_dim, num_layers)
#model.load_state_dict(torch.load('models/transf_expect_2head_2layer'))
#model.to(device)




/home/strike/penv/deep/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [6]:
x,y = gen_data(batch=2)
output = model(x)

In [13]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
s = []
# Обучение модели
num_epochs = 4000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    x,y = gen_data(batch=16)
    output = model(x.to(device))
    
    loss = criterion(output, y.to(device))  # Используем только первый измерение для задачи регрессии
    loss.backward()
    optimizer.step()
    l = loss.detach().to('cpu')
    s.append(l)
    if (epoch + 1) % 100 == 0:
        L = np.mean(s)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {L:.4f}')
        s = []

Epoch [100/4000], Loss: 0.4356
Epoch [200/4000], Loss: 0.4278
Epoch [300/4000], Loss: 0.4218
Epoch [400/4000], Loss: 0.4266
Epoch [500/4000], Loss: 0.4475
Epoch [600/4000], Loss: 0.4374
Epoch [700/4000], Loss: 0.4007
Epoch [800/4000], Loss: 0.3994
Epoch [900/4000], Loss: 0.4186
Epoch [1000/4000], Loss: 0.4186
Epoch [1100/4000], Loss: 0.4368
Epoch [1200/4000], Loss: 0.3938
Epoch [1300/4000], Loss: 0.4391
Epoch [1400/4000], Loss: 0.4264
Epoch [1500/4000], Loss: 0.4302
Epoch [1600/4000], Loss: 0.3814
Epoch [1700/4000], Loss: 0.4001
Epoch [1800/4000], Loss: 0.3999
Epoch [1900/4000], Loss: 0.3975
Epoch [2000/4000], Loss: 0.3896
Epoch [2100/4000], Loss: 0.4270
Epoch [2200/4000], Loss: 0.4348
Epoch [2300/4000], Loss: 0.4220
Epoch [2400/4000], Loss: 0.3974
Epoch [2500/4000], Loss: 0.3908
Epoch [2600/4000], Loss: 0.4018
Epoch [2700/4000], Loss: 0.3883
Epoch [2800/4000], Loss: 0.3854
Epoch [2900/4000], Loss: 0.4045
Epoch [3000/4000], Loss: 0.3782
Epoch [3100/4000], Loss: 0.3922
Epoch [3200/4000]

In [22]:
x,y = gen_data(batch=1)
output = model(x)
output, y

(tensor([[1.4408, 2.5631]], grad_fn=<AddmmBackward0>),
 tensor([[1.5181, 2.2340]]))